In [1]:
%load_ext autoreload
%autoreload 2
import adanet

In [1]:
OUTPUT_DIR = './tmp/adanet'
datadir = './data'
is_tensorboard = True

In [2]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6006 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6006 &')

In [4]:
# this will kill the processes for Tensorboard
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill

# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill

kill: (20788)kill: (20794)

In [5]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)
print(x_train.shape, x_test.shape)

Using TensorFlow backend.


p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3)


In [6]:
import p8_util_config
p8_util_config.ADANET_MAX_ITERATION_STEPS

3

In [7]:
import p8_util_config
import NNGenerator

nn_type = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)

nClasses = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type']

estimator = adanet.Estimator(
    head=p8_util_config.dict_adanet_config['adanet_tf_head'],
    
    subnetwork_generator=NNGenerator.NNGenerator(p8_util_config.dict_adanet_config),
            
    adanet_lambda=p8_util_config.dict_adanet_config['adanet_lambda'],#0.0,#.015,
    
    max_iteration_steps=p8_util_config.ADANET_MAX_ITERATION_STEPS,
    
    evaluator=adanet.Evaluator(
        input_fn=p8_util.input_fn("train", x_train, y_train\
                          , p8_util_config.NUM_EPOCHS
                          , tuple_dimension=tuple_dimension\
                          , batch_size=p8_util_config.BATCH_SIZE),
        steps=None),
    config=  my_config)

*** INFO : MyGenerator : instantiation DONE!
INFO:tensorflow:Using config: {'_model_dir': './tmp/adanet/CNNBase', '_tf_random_seed': 42, '_save_summary_steps': 5, '_save_checkpoints_steps': 5, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbb19c09390>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
import p8_util_config
print("Trains steps= {}".format(p8_util_config.TRAIN_STEPS))
print("MAX Adanet steps= {}".format(p8_util_config.ADANET_MAX_ITERATION_STEPS))

Trains steps= 9
MAX Adanet steps= 3


In [9]:
import time
import tensorflow as tf

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(
        input_fn= p8_util.input_fn("train"
                           , x_train
                           , y_train, p8_util_config.NUM_EPOCHS \
                           , tuple_dimension=tuple_dimension\
                           , batch_size=p8_util_config.BATCH_SIZE),
        max_steps=p8_util_config.TRAIN_STEPS),
    eval_spec=tf.estimator.EvalSpec(
        input_fn= p8_util.input_fn("test"
                                   , x_test
                                   , y_test
                                   , p8_util_config.NUM_EPOCHS\
                                   , tuple_dimension=tuple_dimension\
                                   , batch_size=p8_util_config.BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=1,  
    )
)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 5 or save_checkpoints_secs None.
INFO:tensorflow:Beginning training AdaNet iteration 0
INFO:tensorflow:Calling model_fn.

*** NNAdaNetBuilder : NN Type=CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase
INFO:tensorflow:Building iteration 0
INFO:tensorflow:Building subnetwork 'CNNBase_linear'


*** build_subnetwork() : NN type= CNNBase
INFO:tensorflow:Building subnetwork 'CNNBase_layer_1'


*** build_subnetwork() : NN type= CNNBase
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./tmp/adane

INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNNBase_layer_1/weighted_subnetwork_0/logits/mixture_weight; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNNBase_layer_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/candidate_t0_CNNBase_layer_1/adanet_loss; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/candidate_t0_CNNBase_layer_1/adanet/iteration_0/candidate_t0_CNNBase_layer_1/adanet_loss/biased; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/candidate_t0_CNNBase_layer_1/adanet/iteration_0/candidate_t0_CNNBase_layer_1/adanet_loss/local_step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/train_op/is_over/is_over_var_fn/is_over_var; prev_var_name: Unchanged

*** NNAdaNetBuilder : NN Type=CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase
INFO:tensorflow:Building i

INFO:tensorflow:Building subnetwork 'CNNBase_layer_1'


*** build_subnetwork() : NN type= CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase
INFO:tensorflow:Rebuilding iteration 1
INFO:tensorflow:Building subnetwork 'CNNBase_layer_1'


*** build_subnetwork() : NN type= CNNBase
INFO:tensorflow:Warm-starting from: ('./tmp/adanet/CNNBase/model.ckpt-6',)
INFO:tensorflow:Warm-starting variable: global_step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/step; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNNBase_layer_1/weighted_subnetwork_0/subnetwork/batch_normalization/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNNBase_layer_1/weighted_subnetwork_0/subnetwork/batch_normalization/beta; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: adanet/iteration_0/ensemble_t0_CNNBase_layer_1/weighted_subne

INFO:tensorflow:Finished bookkeeping phase for iteration 1
INFO:tensorflow:Beginning training AdaNet iteration 2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Importing architecture from ./tmp/adanet/CNNBase/architecture-1.txt: ['0:CNNBase_layer_1', '1:CNNBase_layer_1'].

*** NNAdaNetBuilder : NN Type=CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase
INFO:tensorflow:Rebuilding iteration 0
INFO:tensorflow:Building subnetwork 'CNNBase_layer_1'


*** build_subnetwork() : NN type= CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase
INFO:tensorflow:Rebuilding iteration 1
INFO:tensorflow:Building subnetwork 'CNNBase_layer_1'


*** build_subnetwork() : NN type= CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase

*** NNAdaNetBuilder : NN Type=CNNBase
INFO:tensorflow:Building iteration 2
INFO:tensorflow:Building subnetwork 'CNNBase_layer_1'


*** build_subnetwork() : NN type= CNNBase
INFO:tensorflow:Building subnetwork 'CNNBase_layer_2'


*** build_subnetwork() : 

In [10]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("\n{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 99.29052305221558

ACCURACY: 0.38297873735427856

ACCURACY/ADANET/ADANET_WEIGHTED_ENSEMBLE: 0.38297873735427856

ACCURACY/ADANET/SUBNETWORK: 0.38297873735427856

ACCURACY/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 0.38297873735427856

ARCHITECTURE/ADANET/ENSEMBLES: b'\n\x8d\x01\nBadanet/iteration_2/ensemble_t2_CNNBase_layer_1/architecture/adanetB=\x08\x07\x12\x00B7| CNNBase_layer_1 | CNNBase_layer_1 | CNNBase_layer_1 |J\x08\n\x06\n\x04text'

AVERAGE_LOSS: 1.0919487476348877

AVERAGE_LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 1.0919487476348877

AVERAGE_LOSS/ADANET/SUBNETWORK: 1.0948448181152344

AVERAGE_LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 1.0919488668441772

LOSS: 1.0919487476348877

LOSS/ADANET/ADANET_WEIGHTED_ENSEMBLE: 1.0919487476348877

LOSS/ADANET/SUBNETWORK: 1.0948448181152344

LOSS/ADANET/UNIFORM_AVERAGE_ENSEMBLE: 1.0919488668441772

GLOBAL_STEP: 9


In [11]:
#import tensorflow as tf
#help(tf.estimator.Estimator)

In [12]:
def ensemble_architecture(result):
  """Extracts the ensemble architecture from evaluation results."""

  architecture = result["architecture/adanet/ensembles"]
  # The architecture is a serialized Summary proto for TensorBoard.
  summary_proto = tf.summary.Summary.FromString(architecture)
  return summary_proto.value[0].tensor.string_val[0]

In [13]:
print("Architecture: {} / Accuracy= {} /Loss= {}".format(ensemble_architecture(results)\
                                                  , results['accuracy'], results['loss']))

Architecture: b'| CNNBase_layer_1 | CNNBase_layer_1 | CNNBase_layer_1 |' / Accuracy= 0.38297873735427856 /Loss= 1.0919487476348877


In [14]:
# Architecture: b'| DNN_layer_1 | DNN_layer_2 | DNN_layer_3 |' : accuracy = 40% / Loss = 1.0883579
# Architecture: b'| CNNBase_layer_1 |' / Accuracy= 0.40425533056259155 /Loss= 1.081491470336914
# Architecture: b'| CNNBase_linear | CNNBase_layer_1 |' / Accuracy= 0.42553192377090454 /Loss= 1.1250361204147339

In [9]:
[i%2 for i in range(0,10)]

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

In [4]:
1%2

1

In [5]:
2%2

0

In [6]:
3%2

1

In [7]:
4%2

0